In [64]:
import sys
sys.path.append("/home/marin/projects/mhac/build/release")
import mhac

In [65]:
%matplotlib inline
import os, random, copy, math, csv, time
import matplotlib.pyplot as plt
import numpy as np

In [66]:
mhac.set_log_level(mhac.LogLevel.off)

In [67]:
def read_file_input(filepath):
    with open(filepath) as f:
        lines = f.readlines()
        sizes = [int(nr) for nr in lines[0].split()]
        m, n = sizes[0], sizes[1]
        arr = np.zeros((n, m))

        for i in range(1, len(lines)):
            arr[i-1, :] = [int(nr) for nr in lines[i].split()]

    return arr

In [68]:
def nparray_to_timematrix(arr):
    tm = mhac.problems.jss.TimeMatrix()

    for time_list in arr.tolist():
        vint = mhac.VectorInt()
        for duration in time_list:
            vint.append(int(duration))
        tm.append(vint)

    return tm

In [69]:
problem = mhac.problems.jss.JSSP(nparray_to_timematrix(read_file_input("../../data/jss/imrg/ds1/testbed_1_s/t1s_0001.txt")))
TS = mhac.math.TabuSearch(problem)
sol = TS.solve(1000, 10, 20)
sol.schedule

VectorInt[3, 0, 1, 2, 4]

In [70]:
sol.cost

784.0

In [71]:
def read_all_files_in_folder(folder_path):
    # List to hold file contents
    file_contents = {}

    # Check if the provided path is a directory
    if not os.path.isdir(folder_path):
        raise ValueError(f"The provided path '{folder_path}' is not a valid directory.")

    # Iterate over all files in the directory
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Check if it's a file
        if os.path.isfile(file_path):
            try:
                # Open and read the file
                with open(file_path, 'r') as file:
                    content = file.read()
                    file_contents[filename] = content
            except Exception as e:
                print(f"Error reading file {filename}: {e}")
    
    return file_contents

In [86]:
def process_files(file_data, folder_path):
    results = []
    total_time = 0
    for filename, content in file_data.items():
        problem = mhac.problems.jss.JSSP(nparray_to_timematrix(read_file_input(f"{folder_path}/{filename}")))
        SA = mhac.physics.SimulatedAnnealing(problem)

        start_time = time.time()
        sol = SA.solve(100, 0.1, 0.995)
        end_time = time.time()
        duration = end_time - start_time

        print(f"Processing file: {filename} took {duration:.4f}s")

        results.append({"filename": filename, "cost": sol.cost, "time": duration})
        total_time += duration

    print(f"Total time spent solving: {total_time:.4f}s")
    return results

In [73]:
def write_results_to_csv(results, output_file):
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['filename', 'cost', 'time']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for result in results:
            writer.writerow(result)

In [74]:
folder_path = "../../data/jss/imrg/ds1/testbed_1_s"
file_data = read_all_files_in_folder(folder_path)

In [87]:
results = process_files(file_data, folder_path)

Processing file: t1s_0238.txt took 0.0424s
Processing file: t1s_0216.txt took 0.0502s
Processing file: t1s_0312.txt took 0.0300s
Processing file: t1s_0281.txt took 0.0276s
Processing file: t1s_0113.txt took 0.0353s
Processing file: t1s_0082.txt took 0.0175s
Processing file: t1s_0192.txt took 0.0278s
Processing file: t1s_0266.txt took 0.0178s
Processing file: t1s_0021.txt took 0.0154s
Processing file: t1s_0018.txt took 0.0107s
Processing file: t1s_0162.txt took 0.0209s
Processing file: t1s_0147.txt took 0.0133s
Processing file: t1s_0183.txt took 0.0253s
Processing file: t1s_0346.txt took 0.0500s
Processing file: t1s_0062.txt took 0.0164s
Processing file: t1s_0027.txt took 0.0095s
Processing file: t1s_0200.txt took 0.0279s
Processing file: t1s_0111.txt took 0.0232s
Processing file: t1s_0213.txt took 0.0445s
Processing file: t1s_0034.txt took 0.0119s
Processing file: t1s_0295.txt took 0.0204s
Processing file: t1s_0057.txt took 0.0185s
Processing file: t1s_0328.txt took 0.0294s
Processing 

In [89]:
results.sort(key=lambda x: x["filename"])
output_file = "results/mhac.csv"
write_results_to_csv(results, output_file)

# Results

| T | T_min | k | Time |
|---|-------|---|------|
| 1000 | 0.000001 | 0.9995 | 318.20s |
| 1000 | 0.001 | 0.9995 | 239.10s |
| 100 | 0.1 | 0.9995 | 139.87s |
| 100 | 0.1 | 0.999 | 53.26s |
| 100 | 0.1 | 0.995 | 11.30s |

# TODO: compare with PythonTSP

In [77]:
class PythonJSSP(mhac.common.Problem):
    def __init__(self, processing_times):
        super().__init__()
        self.processing_times = processing_times
        self.N = len(processing_times)
        self.M = len(processing_times[0])

    def generateInitialSolution(self):
        sol = mhac.problems.jss.JSSS()
        sol.schedule = random.sample(range(self.N), self.N)
        return sol

    def generateNewSolution(self, initialSol: mhac.problems.jss.JSSS):
        i, j = sorted(random.sample(range(self.N), 2))
        newSol = mhac.problems.jss.JSSS()
        newSol.schedule = initialSol.schedule  # copying with pybind, not using python refs
        # old_segment = newSol.tour[i:j+1]
        # newSol.tour[i:j+1] = old_segment[::-1]  # Slice and reverse the segment
        newSol.schedule[i], newSol.schedule[j] = newSol.schedule[j], newSol.schedule[i]
        return newSol

    def evaluateSolution(self, sol: mhac.problems.jss.JSSS):        
        # Initialize completion times matrix
        completion_times = [[0] * self.M for _ in range(self.N)]
        total_completion_time = 0

        for job_index in range(self.N):
            job = sol.schedule[job_index]

            for machine in range(self.M):
                if job_index == 0:
                    if machine == 0:
                        completion_times[job_index][machine] = self.processing_times[job][machine]
                    else:
                        completion_times[job_index][machine] = completion_times[job_index][machine-1] + self.processing_times[job][machine]
                else:
                    if machine == 0:
                        completion_times[job_index][machine] = completion_times[job_index-1][machine] + self.processing_times[job][machine]
                    else:
                        completion_times[job_index][machine] = self.processing_times[job][machine] + max(completion_times[job_index][machine-1], completion_times[job_index-1][machine])

                if machine == self.M - 1:
                    total_completion_time += completion_times[job_index][machine]

        return total_completion_time

In [88]:
def process_files_pyjssp(file_data):
    results = []
    total_time = 0
    for filename, content in file_data.items():
        problem = PythonJSSP(nparray_to_timematrix(read_file_input(f"{folder_path}/{filename}")))
        # problem = mhac.problems.jss.JSSP(nparray_to_timematrix(read_file_input(f"{folder_path}/{filename}")))
        SA = mhac.physics.SimulatedAnnealing(problem)

        start_time = time.time()
        sol = SA.solve(100, 0.1, 0.995)
        end_time = time.time()
        duration = end_time - start_time

        print(f"Processing file: {filename} took {duration:.4f}s")

        results.append({"filename": filename, "cost": sol.cost, "time": duration})
        total_time += duration

    print(f"Total time spent solving: {total_time:.4f}s")
    return results

In [90]:
results = process_files_pyjssp(file_data)

Processing file: t1s_0238.txt took 0.6421s
Processing file: t1s_0216.txt took 0.6942s
Processing file: t1s_0312.txt took 0.3954s
Processing file: t1s_0281.txt took 0.2366s
Processing file: t1s_0113.txt took 0.3481s
Processing file: t1s_0082.txt took 0.1715s
Processing file: t1s_0192.txt took 0.3584s
Processing file: t1s_0266.txt took 0.1502s
Processing file: t1s_0021.txt took 0.0847s
Processing file: t1s_0018.txt took 0.0732s
Processing file: t1s_0162.txt took 0.2026s
Processing file: t1s_0147.txt took 0.1091s
Processing file: t1s_0183.txt took 0.3440s
Processing file: t1s_0346.txt took 0.7387s
Processing file: t1s_0062.txt took 0.2189s
Processing file: t1s_0027.txt took 0.0837s
Processing file: t1s_0200.txt took 0.3683s
Processing file: t1s_0111.txt took 0.3156s
Processing file: t1s_0213.txt took 0.6242s
Processing file: t1s_0034.txt took 0.1080s
Processing file: t1s_0295.txt took 0.2257s
Processing file: t1s_0057.txt took 0.1148s
Processing file: t1s_0328.txt took 0.3985s
Processing 

# Results

| T | T_min | k | Time |
|---|-------|---|------|
| 1000 | 0.000001 | 0.9995 | 2725.92s |
| 1000 | 0.001 | 0.9995 | 1751.22s |
| 100 | 0.1 | 0.9995 | 876.53s |
| 100 | 0.1 | 0.999 | 500.27s |
| 100 | 0.1 | 0.995 | 90.68s |
